## Import Modules

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
from ble_rx_stream import *
import time
import numpy as np
import matplotlib.pyplot as plt

LOG.propagate = False

## Connect to Artemis

In [ ]:
ble = get_ble_controller()
ble.connect()

## Testing

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.STREAM_TOF1, "1000")
tof1_data = np.array(list(unpack_stream('LH*', await fut)))
print(tof1_data)
plt.plot(tof1_data[:, 0], tof1_data[:, 1])

In [ ]:
ble.send_command(CMD.DRIVE, "1000|100|0|100|0")

## Run PID

In [ ]:
time.sleep(5)
timeout_s = 8
target_ft = 1
deadband = 40
calibration = 1
K_p = -0.05
K_i = 0
K_d = 0
ble.send_command(CMD.RUN_PID, f"{int(timeout_s*1000)}|{int(target_ft*304)}|{deadband}|{calibration}|{K_p}|{K_i}|{K_d}")
time.sleep(timeout_s)

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.DATA_PID, "")
stream_data = list(unpack_stream('LHfBB*', await fut))

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.EXTRA_DATA_PID, "")
extra_stream_data = list(unpack_stream('Lfff*', await fut))

In [ ]:
data = np.array(stream_data)
data_time = data[:, 0]
data_dist = data[:, 1]
data_pid = data[:, 2]
data_mot = data[:, 3:5]
extra_data = np.array(extra_stream_data)
extra_time = extra_data[:, 0]
extra_err = extra_data[:, 1]
extra_int = extra_data[:, 2]
extra_dif = extra_data[:, 3]

In [ ]:
_, axs = plt.subplots(2, 1, sharex=True)
axs[0].plot(data_time, data_dist)
axs[0].axhline(target_ft*304, c='red', ls='--')
axs[0].set_ylabel('distance (mm)')
axs[1].plot(data_time, data_mot)
axs[1].set_ylabel('PWM output')
plt.xlabel('time (ms)')
plt.savefig('run.jpg')

In [ ]:
_, axs = plt.subplots(6, 1, sharex=True)
axs[0].plot(data_time, data_dist)
axs[0].set_ylabel('distance (mm)')
axs[1].plot(data_time, data_pid)
axs[1].set_ylabel('raw PID output')
axs[2].plot(data_time, data_mot, label=['motor 1', 'motor 2'])
axs[2].set_ylabel('motor PWM output')
axs[2].legend()
axs[3].plot(extra_time, extra_err)
axs[3].set_ylabel('PID: error')
axs[4].plot(extra_time, extra_int)
axs[4].set_ylabel('PID: integrator')
axs[5].plot(extra_time, extra_dif)
axs[5].set_ylabel('PID: differentiator')
plt.xlabel('time (ms)')
plt.show()

## Disconnect from Artemis

In [ ]:
ble.disconnect()